In [1]:
import seaborn as sns
import pandas as pd
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder, StandardScaler, Normalizer, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import janitor
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [3]:
train, val = train_test_split(train, random_state = 42, train_size = 0.8, stratify = train["attrition_flag"])

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6480 entries, 6694 to 3648
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   clientnum                 6480 non-null   int64  
 1   attrition_flag            6480 non-null   object 
 2   customer_age              6480 non-null   int64  
 3   gender                    6480 non-null   object 
 4   dependent_count           6480 non-null   int64  
 5   education_level           6480 non-null   object 
 6   marital_status            6480 non-null   object 
 7   income_category           6480 non-null   object 
 8   card_category             6480 non-null   object 
 9   months_on_book            6480 non-null   int64  
 10  total_relationship_count  6480 non-null   int64  
 11  months_inactive_12_mon    6480 non-null   int64  
 12  contacts_count_12_mon     6480 non-null   int64  
 13  credit_limit              6480 non-null   float64
 14  total

In [5]:
train.head()

,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,...,months_inactive_12_mon,contacts_count_12_mon,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio
6694,716389833,Existing Customer,50,F,1,Unknown,Single,Less than $40K,Blue,40,...,1,2,2319.0,1026,1293.0,0.521,4099,74,0.721,0.442
6698,787430733,Existing Customer,56,F,2,High School,Married,$40K - $60K,Blue,41,...,3,2,2771.0,1446,1325.0,0.686,1706,27,0.227,0.522
5560,719539458,Existing Customer,46,F,3,Graduate,Single,Less than $40K,Blue,41,...,3,3,3913.0,0,3913.0,0.722,5170,93,0.860,0.000
2893,716791908,Existing Customer,43,M,1,College,Married,$80K - $120K,Blue,23,...,2,1,14566.0,1045,13521.0,0.417,4480,77,0.791,0.072
796,709562358,Existing Customer,55,F,2,Unknown,Single,Less than $40K,Blue,43,...,1,5,5353.0,1594,3759.0,0.973,4456,79,0.837,0.298


### Hardcoded Feature Engineering

In [6]:
train["revolving_trans"] = train["total_revolving_bal"]/train["total_trans_amt"]
val["revolving_trans"] = val["total_revolving_bal"]/val["total_trans_amt"]
test["revolving_trans"] = test["total_revolving_bal"]/test["total_trans_amt"]

In [7]:
train["marriage_education"] = train["marital_status"] + "_" + train["education_level"]
val["marriage_education"] = val["marital_status"] + "_" + val["education_level"]
test["marriage_education"] = test["marital_status"] + "_" + test["education_level"]

### Feature Engineering via Pipeline

In [8]:
one_hot = OneHotEncoder()
min_max = MinMaxScaler()
sd = StandardScaler()
le = LabelEncoder()

### Individual Feature Engineering

In [9]:
ct = ColumnTransformer([("ohe", one_hot, ["marital_status"])])

In [10]:
x = ct.fit_transform(train).toarray()

In [11]:
train_new = train.copy()

In [12]:
element = sorted(train_new["marital_status"].unique())
element[0].lower()
for i in range(len(element)):
    train_new[element[i].lower()] = x[:,i]
train_new = train_new.drop("marital_status", axis = 1)

In [13]:
train_new.head()

,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,income_category,card_category,months_on_book,total_relationship_count,...,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,revolving_trans,marriage_education,divorced,married,single,unknown
6694,716389833,Existing Customer,50,F,1,Unknown,Less than $40K,Blue,40,4,...,4099,74,0.721,0.442,0.250305,Single_Unknown,0.0,0.0,1.0,0.0
6698,787430733,Existing Customer,56,F,2,High School,$40K - $60K,Blue,41,5,...,1706,27,0.227,0.522,0.847597,Married_High School,0.0,1.0,0.0,0.0
5560,719539458,Existing Customer,46,F,3,Graduate,Less than $40K,Blue,41,5,...,5170,93,0.860,0.000,0.000000,Single_Graduate,0.0,0.0,1.0,0.0
2893,716791908,Existing Customer,43,M,1,College,$80K - $120K,Blue,23,5,...,4480,77,0.791,0.072,0.233259,Married_College,0.0,1.0,0.0,0.0
796,709562358,Existing Customer,55,F,2,Unknown,Less than $40K,Blue,43,6,...,4456,79,0.837,0.298,0.357720,Single_Unknown,0.0,0.0,1.0,0.0


In [14]:
## One Hot Encoding Function
def one_hot_func(df, var):
    df_use = df.copy()
    element = sorted(df_use[var].unique())
    element[0].lower()
    ct = ColumnTransformer([("ohe", one_hot, [var])])
    x = ct.fit_transform(df).toarray()
    for i in range(len(element)):
        df_use[var + "_" + element[i].lower()] = x[:,i]
    df_use = df_use.drop(var, axis = 1)
    return df_use

In [15]:
train_update = one_hot_func(train, "marital_status")

In [16]:
train_update.head()

,clientnum,attrition_flag,customer_age,gender,dependent_count,education_level,income_category,card_category,months_on_book,total_relationship_count,...,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,revolving_trans,marriage_education,marital_status_divorced,marital_status_married,marital_status_single,marital_status_unknown
6694,716389833,Existing Customer,50,F,1,Unknown,Less than $40K,Blue,40,4,...,4099,74,0.721,0.442,0.250305,Single_Unknown,0.0,0.0,1.0,0.0
6698,787430733,Existing Customer,56,F,2,High School,$40K - $60K,Blue,41,5,...,1706,27,0.227,0.522,0.847597,Married_High School,0.0,1.0,0.0,0.0
5560,719539458,Existing Customer,46,F,3,Graduate,Less than $40K,Blue,41,5,...,5170,93,0.860,0.000,0.000000,Single_Graduate,0.0,0.0,1.0,0.0
2893,716791908,Existing Customer,43,M,1,College,$80K - $120K,Blue,23,5,...,4480,77,0.791,0.072,0.233259,Married_College,0.0,1.0,0.0,0.0
796,709562358,Existing Customer,55,F,2,Unknown,Less than $40K,Blue,43,6,...,4456,79,0.837,0.298,0.357720,Single_Unknown,0.0,0.0,1.0,0.0


### Multiple Feature Engineering

In [17]:
ct = ColumnTransformer([("ohe", one_hot, ["marital_status","gender","income_category"]),("min_max",min_max,["total_revolving_bal","credit_limit"])])

In [18]:
new_arr = ct.fit_transform(train)

In [19]:
new_arr

array([[0.        , 0.        , 1.        , ..., 0.        , 0.40762813,
        0.02662519],
       [0.        , 1.        , 0.        , ..., 0.        , 0.57449344,
        0.04028998],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.07481475],
       ...,
       [0.        , 1.        , 0.        , ..., 1.        , 0.52403655,
        0.19117714],
       [0.        , 0.        , 1.        , ..., 0.        , 0.51370679,
        0.01241622],
       [0.        , 1.        , 0.        , ..., 0.        , 0.41835518,
        0.01565103]])

In [20]:
new_arr.shape

(6480, 14)

In [21]:
import numpy as np

In [22]:
train["gender"].unique()

array(['F', 'M'], dtype=object)

In [23]:
#columns = ["marital_status", "gender", "income_category", "total_revolving_bal","credit_limit"]
columns = ["marital_status", "gender", "income_category", "total_revolving_bal","credit_limit"]

new_df = pd.DataFrame(index = range(train.shape[0]))
for col in columns:
    if is_numeric_dtype(train[col]):
        ct = ColumnTransformer([("min_max", min_max, [col])])
        new_df[col + "_scaled"] = ct.fit_transform(train)
        #df = df.drop(col, axis = 1)
    elif is_string_dtype(train[col]):
        elements = sorted(train[col].unique())
        ct2 = ColumnTransformer([("ohe", one_hot, [col])])
        if len(elements) > 2:
            x = ct2.fit_transform(train).toarray()
        elif len(elements) <= 2:
            x = np.array(ct2.fit_transform(train))
        for i in range(len(elements)):
            new_df[col + "_" + elements[i].lower()] = x[:,i]
        #df = df.drop(col, axis = 1)

In [24]:
new_df

,marital_status_divorced,marital_status_married,marital_status_single,marital_status_unknown,gender_f,gender_m,income_category_$120k +,income_category_$40k - $60k,income_category_$60k - $80k,income_category_$80k - $120k,income_category_less than $40k,income_category_unknown,total_revolving_bal_scaled,credit_limit_scaled
0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.407628,0.026625
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.574493,0.040290
2,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.074815
3,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.415177,0.396875
4,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.633294,0.118349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6475,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.000000,0.064445
6476,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.509549
6477,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.524037,0.191177
6478,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.513707,0.012416


### Feature Engineering on all features

In [25]:
train_use = train.drop(["clientnum", "attrition_flag"], axis = 1)
train_use.head()

,customer_age,gender,dependent_count,education_level,marital_status,income_category,card_category,months_on_book,total_relationship_count,months_inactive_12_mon,...,credit_limit,total_revolving_bal,avg_open_to_buy,total_amt_chng_q4_q1,total_trans_amt,total_trans_ct,total_ct_chng_q4_q1,avg_utilization_ratio,revolving_trans,marriage_education
6694,50,F,1,Unknown,Single,Less than $40K,Blue,40,4,1,...,2319.0,1026,1293.0,0.521,4099,74,0.721,0.442,0.250305,Single_Unknown
6698,56,F,2,High School,Married,$40K - $60K,Blue,41,5,3,...,2771.0,1446,1325.0,0.686,1706,27,0.227,0.522,0.847597,Married_High School
5560,46,F,3,Graduate,Single,Less than $40K,Blue,41,5,3,...,3913.0,0,3913.0,0.722,5170,93,0.860,0.000,0.000000,Single_Graduate
2893,43,M,1,College,Married,$80K - $120K,Blue,23,5,2,...,14566.0,1045,13521.0,0.417,4480,77,0.791,0.072,0.233259,Married_College
796,55,F,2,Unknown,Single,Less than $40K,Blue,43,6,1,...,5353.0,1594,3759.0,0.973,4456,79,0.837,0.298,0.357720,Single_Unknown


In [26]:
num = []
cat = []
columns = list(train_use.columns)
for col in columns:
    if is_numeric_dtype(train_use[col]):
        num.append(col)
    elif is_string_dtype(train_use[col]):
        cat.append(col)

In [27]:
num

['customer_age',
 'dependent_count',
 'months_on_book',
 'total_relationship_count',
 'months_inactive_12_mon',
 'contacts_count_12_mon',
 'credit_limit',
 'total_revolving_bal',
 'avg_open_to_buy',
 'total_amt_chng_q4_q1',
 'total_trans_amt',
 'total_trans_ct',
 'total_ct_chng_q4_q1',
 'avg_utilization_ratio',
 'revolving_trans']

In [28]:
cat

['gender',
 'education_level',
 'marital_status',
 'income_category',
 'card_category',
 'marriage_education']

In [29]:
ct3 = ColumnTransformer([("min_max", min_max, num), ("one_hot", one_hot, cat)])

In [30]:
train_arr = ct3.fit_transform(train)

In [31]:
train_arr.shape

(6480, 66)

In [32]:
train_arr

array([[0.5106383 , 0.2       , 0.62790698, ..., 0.        , 0.        ,
        0.        ],
       [0.63829787, 0.4       , 0.65116279, ..., 0.        , 0.        ,
        0.        ],
       [0.42553191, 0.6       , 0.65116279, ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.34042553, 0.6       , 0.41860465, ..., 0.        , 0.        ,
        0.        ],
       [0.36170213, 0.4       , 0.51162791, ..., 0.        , 0.        ,
        0.        ],
       [0.31914894, 0.6       , 0.53488372, ..., 0.        , 0.        ,
        0.        ]])

In [33]:
def column_transform(df, columns):
    new_df = df.copy()
    for col in columns:
        if is_numeric_dtype(df[col]):
            ct = ColumnTransformer([("min_max", min_max, [col])])
            new_df[col + "_scaled"] = ct.fit_transform(df)
        elif is_string_dtype(df[col]):
            elements = sorted(df[col].unique())
            ct2 = ColumnTransformer([("ohe", one_hot, [col])])
            if len(elements) > 2:
                x = ct2.fit_transform(df).toarray()
            elif len(elements) <= 2:
                x = np.array(ct2.fit_transform(df))
            for i in range(len(elements)):
                new_df[col + "_" + elements[i].lower()] = x[:,i]
    new_df = new_df.drop(columns, axis = 1)
    return new_df

In [34]:
def column_fit_transform(train, other, columns):
    new_df = other.copy()
    for col in columns:
        if is_numeric_dtype(other[col]):
            ct = ColumnTransformer([("min_max", min_max, [col])])
            ct.fit(train)
            new_df[col + "_scaled"] = ct.transform(other)
        elif is_string_dtype(other[col]):
            elements = sorted(other[col].unique())
            ct2 = ColumnTransformer([("ohe", one_hot, [col])])
            if len(elements) > 2:
                ct2.fit(train)
                x = ct2.transform(other).toarray()
            elif len(elements) <= 2:
                ct2.fit(train)
                x = np.array(ct2.transform(other))
            for i in range(len(elements)):
                new_df[col + "_" + elements[i].lower()] = x[:,i]
    new_df = new_df.drop(columns, axis = 1)
    return new_df

#### Transforming Training data

In [35]:
train_renewed = column_transform(train_use, list(train_use.columns))
train_renewed["attrition_flag"] = train["attrition_flag"].replace(["Attrited Customer","Existing Customer"], [1,0]).astype(int)
train_renewed["clientnum"] = train["clientnum"]
train_renewed = train_renewed.clean_names()
train_renewed.head()

,customer_age_scaled,gender_f,gender_m,dependent_count_scaled,education_level_college,education_level_doctorate,education_level_graduate,education_level_high_school,education_level_post_graduate,education_level_uneducated,...,marriage_education_single_unknown,marriage_education_unknown_college,marriage_education_unknown_doctorate,marriage_education_unknown_graduate,marriage_education_unknown_high_school,marriage_education_unknown_post_graduate,marriage_education_unknown_uneducated,marriage_education_unknown_unknown,attrition_flag,clientnum
6694,0.510638,1.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,716389833
6698,0.638298,1.0,0.0,0.4,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,787430733
5560,0.425532,1.0,0.0,0.6,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,719539458
2893,0.361702,0.0,1.0,0.2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,716791908
796,0.617021,1.0,0.0,0.4,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,709562358


In [36]:
train_renewed.to_csv("data/train_fe.csv", index = False)

#### Transforming validation data

In [37]:
val_use = val.drop(["clientnum", "attrition_flag"], axis = 1)
val_renewed = column_fit_transform(train_use, val_use, list(val_use.columns))
val_renewed["attrition_flag"] = val["attrition_flag"].replace(["Attrited Customer","Existing Customer"], [1,0]).astype(int)
val_renewed["clientnum"] = val["clientnum"]
val_renewed = val_renewed.clean_names()
val_renewed.head()

,customer_age_scaled,gender_f,gender_m,dependent_count_scaled,education_level_college,education_level_doctorate,education_level_graduate,education_level_high_school,education_level_post_graduate,education_level_uneducated,...,marriage_education_single_unknown,marriage_education_unknown_college,marriage_education_unknown_doctorate,marriage_education_unknown_graduate,marriage_education_unknown_high_school,marriage_education_unknown_post_graduate,marriage_education_unknown_uneducated,marriage_education_unknown_unknown,attrition_flag,clientnum
5423,0.489362,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,788916933
4059,0.361702,1.0,0.0,0.8,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0,711452583
1076,0.468085,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,710780883
1295,0.553191,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,719326758
4485,0.255319,1.0,0.0,0.2,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,708907833


In [38]:
val_renewed.to_csv("data/val_fe.csv", index = False)

#### Transforming test data

In [39]:
test_use = test.drop(["clientnum", "attrition_flag"], axis = 1)
test_renewed = column_fit_transform(train_use, test_use, list(test_use.columns))
test_renewed["attrition_flag"] = test["attrition_flag"].replace(["Attrited Customer","Existing Customer"], [1,0]).astype(int)
test_renewed["clientnum"] = test["clientnum"]
test_renewed = test_renewed.clean_names()
test_renewed.head()

,customer_age_scaled,gender_f,gender_m,dependent_count_scaled,education_level_college,education_level_doctorate,education_level_graduate,education_level_high_school,education_level_post_graduate,education_level_uneducated,...,marriage_education_single_unknown,marriage_education_unknown_college,marriage_education_unknown_doctorate,marriage_education_unknown_graduate,marriage_education_unknown_high_school,marriage_education_unknown_post_graduate,marriage_education_unknown_uneducated,marriage_education_unknown_unknown,attrition_flag,clientnum
0,0.446809,0.0,1.0,0.6,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,720943308
1,0.531915,0.0,1.0,0.2,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,710879283
2,0.617021,0.0,1.0,0.4,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,787800108
3,0.297872,1.0,0.0,0.8,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,789200208
4,0.765957,0.0,1.0,0.2,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,711332883


In [40]:
test_renewed.to_csv("data/test_fe.csv", index = False)